# Idiomatic Pandas
## 5 Tips for Better Pandas Code

https://github.com/mattharrison/2023-scipy-pandas

## About Matt  Harrison @\_\_mharrison\_\_

* Author of Effective Pandas, Machine Learning Pocket Reference, and Illustrated Guide to Python 3.
* Advisor at Ponder (creators of Modin)
* Corporate trainer at MetaSnake. Taught Pandas to 1000's of students.

## Practice this on your data with your team!
* Contact me matt@metasnake.com
* Follow on Twitter @\_\_mharrison\_\_

## Outline

* Load Data
* Types
* Chaining
* Mutation
* Apply
* Aggregation

## Imports

In [ ]:
%matplotlib inline
from IPython.display import display
import numpy as np
import pandas as pd
import pyarrow

import io
import zipfile
#import modin.pandas as pd

In [ ]:
pd.__version__

In [ ]:
pyarrow.__version__

## Data

Don't need to run this, but this is how I created the data

In [ ]:
# https://gss.norc.org/get-the-data/stata
# takes a few minutes on my computer to load
path = '~/Downloads/gss_spss_with_codebook.zip'
with zipfile.ZipFile(path) as z:
    print(z.namelist())
    with open('gss.sav', mode='bw') as fout:
        fout.write(z.open('GSS7218_R3.sav').read())
    gss = pd.read_spss('gss.sav')

In [ ]:
!pip install pyreadstat

In [ ]:
%%time
import pyreadstat
gss, meta = pyreadstat.read_sav('gss.sav')

In [ ]:
gss.shape

In [ ]:
gss.to_feather('gss.fth')

In [ ]:
%%time
raw = pd.read_feather('~/Dropbox/work/jupyter/gss.fth')

In [ ]:
raw

In [ ]:
# 6000 columns!
raw.shape

In [ ]:
cols = ['YEAR','ID','AGE', 'HRS1','OCC','MAJOR1','SEX','RACE','BORN','INCOME',
        'INCOME06','HONEST','TICKET']

raw[cols].to_feather('honest.fth')

## Loading Data

In [ ]:
raw = pd.read_feather('honest.fth', dtype_backend='pyarrow')

## My Cleanup
See GSS_Codebook.pdf for explanation

Columns:

* YEAR
* ID - RESPONDENT ID NUMBER
* AGE - AGE OF RESPONENT
* HRS1 - NUMBER OF HOURS WORKED LAST WEEK
* OCC - R'S CENSUS OCCUPATION CODE (1970) - Page 126 (VAR: OCC) see page 125 for notes APPENDIX F,G,H
   Appendix F - Page 3286
* MAJOR1 - COLLEGE MAJOR 1
* SEX - RESPONDENTS SEX
* RACE - RACE OF RESPONDENT
* BORN -  WAS R BORN IN THIS COUNTRY
* INCOME - TOTAL FAMILY INCOME 1970
* INCOME06 - TOTAL FAMILY INCOME 2006
* HONEST - HONEST
* TICKET - EVER RECEIVED A TRAFFIC TICKET


In [ ]:
cols = ['YEAR','ID','AGE', 'HRS1','OCC','MAJOR1','SEX','RACE','BORN','INCOME',
        'INCOME06','HONEST','TICKET']

raw[cols].isna().mean()*100

In [ ]:
(raw
 [cols]
 .isna()
 .mean()*100
)

In [ ]:
MAJOR= '''RESPONSE PUNCH 1972-82 1982B 1983-87 1987B 1988-91 1993-98 2000-04 2006 2008 2010 2012 2014 2016 2018 ALL
Accounting/bookkeeping 1 0 0 0 0 0 0 0 0 0 0 28 32 30 29 119
Advertising 2 0 0 0 0 0 0 0 0 0 0 3 2 0 0 5
Agriculture/horticulture 3 0 0 0 0 0 0 0 0 0 0 8 2 7 5 22
Allied health 4 0 0 0 0 0 0 0 0 0 0 0 2 1 0 3
Anthropology 5 0 0 0 0 0 0 0 0 0 0 3 5 1 1 10
Architecture 6 0 0 0 0 0 0 0 0 0 0 2 3 5 3 13
Art 7 0 0 0 0 0 0 0 0 0 0 6 7 11 10 34
Biology 8 0 0 0 0 0 0 0 0 0 0 16 22 33 26 97
Business administration 9 0 0 0 0 0 0 0 0 0 0 90 142 172 138 542
Chemistry 11 0 0 0 0 0 0 0 0 0 0 5 8 10 4 27
Communications/speech 12 0 0 0 0 0 0 0 0 0 0 20 18 26 18 82
Comm. disorders 13 0 0 0 0 0 0 0 0 0 0 4 6 2 2 14
Computer science 14 0 0 0 0 0 0 0 0 0 0 25 24 33 17 99
Dentistry 15 0 0 0 0 0 0 0 0 0 0 2 4 3 5 14
Education 16 0 0 0 0 0 0 0 0 0 0 73 91 97 79 340
Economics 17 0 0 0 0 0 0 0 0 0 0 11 19 13 19 62
Engineering 18 0 0 0 0 0 0 0 0 0 0 47 49 47 54 197
English 19 0 0 0 0 0 0 0 0 0 0 23 26 27 24 100
Finance 20 0 0 0 0 0 0 0 0 0 0 7 15 14 16 52
Foreign language 21 0 0 0 0 0 0 0 0 0 0 4 8 6 5 23
Forestry 22 0 0 0 0 0 0 0 0 0 0 1 0 3 0 4
Geography 23 0 0 0 0 0 0 0 0 0 0 0 2 2 4 8
Geology 24 0 0 0 0 0 0 0 0 0 0 1 3 4 2 10
History 25 0 0 0 0 0 0 0 0 0 0 10 19 14 19 62
Home economics 26 0 0 0 0 0 0 0 0 0 0 0 0 3 2 5
Industry & techn 27 0 0 0 0 0 0 0 0 0 0 3 4 6 0 13
Journalism 28 0 0 0 0 0 0 0 0 0 0 5 6 6 4 21
Law 29 0 0 0 0 0 0 0 0 0 0 13 18 23 14 68
Law enforcement 30 0 0 0 0 0 0 0 0 0 0 3 5 4 2 14
Library science 31 0 0 0 0 0 0 0 0 0 0 4 5 2 3 14
Marketing 32 0 0 0 0 0 0 0 0 0 0 11 15 13 12 51
Mathematics 33 0 0 0 0 0 0 0 0 0 0 5 10 12 5 32
Medicine 34 0 0 0 0 0 0 0 0 0 0 9 25 12 11 57
Music 35 0 0 0 0 0 0 0 0 0 0 4 2 10 2 18
Nursing 36 0 0 0 0 0 0 0 0 0 0 36 39 60 51 186
Optometry 37 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
Pharmacy 38 0 0 0 0 0 0 0 0 0 0 2 5 4 4 15
Philosophy 39 0 0 0 0 0 0 0 0 0 0 2 0 2 2 6
Physical education 40 0 0 0 0 0 0 0 0 0 0 9 6 16 6 37
Physics 41 0 0 0 0 0 0 0 0 0 0 3 6 7 4 20
Psychology 42 0 0 0 0 0 0 0 0 0 0 32 32 34 29 127
Political science/international relations 43 0 0 0 0 0 0 0 0 0 0 16 22 19 14 71
Sociology 44 0 0 0 0 0 0 0 0 0 0 9 15 10 12 46
Special education 45 0 0 0 0 0 0 0 0 0 0 5 3 5 2 15
Theater arts 46 0 0 0 0 0 0 0 0 0 0 6 2 3 1 12
Theology 47 0 0 0 0 0 0 0 0 0 0 6 6 13 8 33
Veterinary medicine 48 0 0 0 0 0 0 0 0 0 0 1 5 3 4 13
Liberal arts 49 0 0 0 0 0 0 0 0 0 0 8 16 12 10 46
Other 50 0 0 0 0 0 0 0 0 0 0 8 10 21 27 66
General sciences 51 0 0 0 0 0 0 0 0 0 0 10 13 15 14 52
Social work 52 0 0 0 0 0 0 0 0 0 0 7 17 24 7 55
General studies 53 0 0 0 0 0 0 0 0 0 0 2 5 7 7 21
Other vocational 54 0 0 0 0 0 0 0 0 0 0 5 11 6 5 27
Health 55 0 0 0 0 0 0 0 0 0 0 23 31 31 42 127
Industrial Relations 56 0 0 0 0 0 0 0 0 0 0 1 0 0 3 4
Child/Human/Family Development 57 0 0 0 0 0 0 0 0 0 0 11 3 7 7 28
Food Science/Nutrition/Culinary Arts 58 0 0 0 0 0 0 0 0 0 0 3 6 9 9 27
Environmental Science/Ecology 59 0 0 0 0 0 0 0 0 0 0 5 5 6 8 24
Social Sciences 60 0 0 0 0 0 0 0 0 0 0 4 2 7 5 18
Human Services/Human Resources 61 0 0 0 0 0 0 0 0 0 0 3 7 7 5 22
Visual Arts/Graphic Design/Design and Drafting 62 0 0 0 0 0 0 0 0 0 0 3 8 9 10 30
Fine Arts 63 0 0 0 0 0 0 0 0 0 0 4 5 5 6 20
Humanities 64 0 0 0 0 0 0 0 0 0 0 0 2 0 1 3
Ethnic studies 65 0 0 0 0 0 0 0 0 0 0 3 1 0 0 4
Educational administration 66 0 0 0 0 0 0 0 0 0 0 3 4 8 9 24
Television/Film 67 0 0 0 0 0 0 0 0 0 0 0 2 6 1 9
Aviation/Aeronatics 68 0 0 0 0 0 0 0 0 0 0 2 1 1 3 7
Statistics/Biostatistics 69 0 0 0 0 0 0 0 0 0 0 0 0 2 2 4
Criminology/Criminal Justice 70 0 0 0 0 0 0 0 0 0 0 13 17 17 13 60
Administrative Science/Public Administration 71 0 0 0 0 0 0 0 0 0 0 2 11 3 5 21
Electronics 72 0 0 0 0 0 0 0 0 0 0 6 6 5 9 26
Urban and Regional Planning 73 0 0 0 0 0 0 0 0 0 0 1 1 3 2 7
Mechanics/Machine Trade 74 0 0 0 0 0 0 0 0 0 0 0 1 1 4 6
Dance 75 0 0 0 0 0 0 0 0 0 0 1 0 1 1 3
Gerontology 76 0 0 0 0 0 0 0 0 0 0 1 0 1 1 3
Public Relations 77 0 0 0 0 0 0 0 0 0 0 3 1 2 1 7
Textiles/Cloth 78 0 0 0 0 0 0 0 0 0 0 3 4 0 0 7
Parks and Recreation 79 0 0 0 0 0 0 0 0 0 0 1 2 1 0 4
Information Technology 80 0 0 0 0 0 0 0 0 0 0 0 5 8 11 24
Fashion 81 0 0 0 0 0 0 0 0 0 0 0 0 3 1 4
Counseling 82 0 0 0 0 0 0 0 0 0 0 0 0 11 9 20
Don't know/UNCODED 98 0 0 0 0 0 0 0 0 0 0 2 3 0 0 5
No answer 99 0 0 0 0 0 0 0 0 0 0 0 1 5 3 9
Not applicable 0 13626 354 7542 353 5907 10334 8394 4510 2023 2044 1263 1597 1795 1435 61177'''

# copy paste slight tweak from page 186
major_dict = {int(row.split()[-16]): ' '.join(row.split()[:-16])  for row in MAJOR.split('\n')[1:]}
major_dict

In [ ]:
raw.MAJOR1.value_counts()

In [ ]:
(raw
 [cols]
 .assign(
     MAJOR1=raw.MAJOR1.fillna(99).astype('int').replace(major_dict),
     SEX=raw.SEX#
           
           .astype(int)
           .replace({1:'Male', 2:'Female'}),
     RACE=raw.RACE.astype(int).replace({1:'White', 2:'Black', 3:'Other'}),
     OCC=raw.OCC.fillna(9999).astype(int),
     BORN=raw.BORN.fillna(4).astype(int).replace({1:'Yes', 2:'No', 3:'Don\'t Know',
                                                    4:'No answer', 5:'Not applicable'}),
     INCOME=raw.INCOME.fillna(99).astype(int).replace({99:'No answer', **dict(enumerate(['Not applicable',
                                                                                  0,1000,3000,4000,5000,6000,
                                                                                  7000,8000,10000,15000,20000,25000,]))}),
     INCOME06=raw.INCOME06.fillna(26).astype(int).replace({26:'Refused', **dict(enumerate(['Not applicable',
                                                                                  0,1000,3000,4000,5000,6000,
                                                                                  7000,8000,10000,12500,15000,
                                                                                  17500,20000,22500,25000,30_000,
                                                                                  35_000, 40_000, 50_000, 60_000,
                                                                                 75_000, 90_000, 110_000, 130_000,
                                                                                 150_000]))}),
     HONEST=raw.HONEST.fillna(9).astype(int).replace({1:'Most desirable', 2:'3 most desireable',
                                                                   3:'Not mentioned', 4:  '3 least desireable',
                                                                   5: 'One least desireable',
                                                                    9:'No answer'}),
     TICKET=raw.TICKET.fillna(9).astype(int).replace({1:'Yes', 2:'No', 3:'Refused', 9: 'No answer'}),
     )
 .astype({'YEAR':int, 'ID': 'uint16[pyarrow]'})
 .to_csv('GSS.csv')
)

## Types
Getting the right types will enable analysis and correctness.


In [ ]:
%%time
gss = pd.read_csv('GSS.csv', index_col=0, dtype_backend='pyarrow', engine='pyarrow')

In [ ]:
gss.dtypes

In [ ]:
gss

In [ ]:
gss.memory_usage(deep=True)

In [ ]:
# 36 M (pandas 1)
# 8.6 M (Pandas 2)
gss.memory_usage(deep=True).sum()

## Ints

In [ ]:
gss.select_dtypes(int).describe()

In [ ]:
# chaining
(gss
 .select_dtypes(int)
 .describe()
)

In [ ]:
# can comb08 be an int8?
# Do completion on int
np.iinfo(np.int)

In [ ]:
np.iinfo(np.uint8)

In [ ]:
np.iinfo(np.uint16)

In [ ]:
# chaining
(gss
 .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]' })
 .select_dtypes(['uint16'])
 .describe()
)

In [ ]:
# chaining
# use 'integer' so see all int-like columns
(gss
 .astype({#'YEAR': 'uint16[pyarrow]',
          'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]' }) 
 .select_dtypes(['integer'])  # see https://numpy.org/doc/stable/reference/arrays.scalars.html
 .describe()
)

In [ ]:
# Inspect memory usage
(gss
 .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]' }) 
 .memory_usage(deep=True)
 .sum()  # was 36M
)

## Int Exercise
* Try converting *YEAR* to `'int8'`. What do the values look like?
* Try converting *YEAR* to `'int8[pyarrow]'`. What do the values look like?

## Floats

In [ ]:
(gss
.select_dtypes('float'))

In [ ]:
# surprise! age and hours worked looks int-like
gss.HRS1.describe()

In [ ]:
# opps! missing values
gss.HRS1.value_counts(dropna=False)

In [ ]:
# where are they missing?
(gss
  .query('HRS1.isna()')
)

In [ ]:
# where are they missing?
(gss
  .query('AGE.isna()')
)

In [ ]:
# where are they missing?
# It turns out that ID is not consistent across years
(gss
  .query('ID == 229')
)

In [ ]:
# Convert to integers
(gss
  .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
         'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]'})
)

In [ ]:
(gss
  .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
         'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]'})
 .memory_usage(deep=True)
 .sum()  # was 36M  
)

## Float Exercise

* What is the mean of the numeric columns?
* How many values are missing in the numeric columns?

## Objects

In [ ]:
# pandas 1.x
(gss
 .select_dtypes(object)
)

In [ ]:
# pandas 2
(gss
 .select_dtypes('string') # str doesn't work
)

In [ ]:
# My goto method - .value_counts
# looks categorical
(gss.MAJOR1.value_counts(dropna=False))

In [ ]:
(gss
  .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
         'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
         'MAJOR1': 'category'})
 .memory_usage(deep=True)
 .sum()  # was 36M  
)

In [ ]:
(gss
 .select_dtypes(object)
 .columns
)

In [ ]:
# wow!
(gss
  .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
         'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
         'MAJOR1': 'category',
          **{col: 'category' for col in ['SEX', 'RACE', 'BORN', 
                'INCOME', 'INCOME06', 'HONEST','TICKET']}})           
 .memory_usage(deep=True)
 .sum()  # was 36M  
)

## String and Category Exercises
* There is a `.cat` attribute on the category columns. What can you do with this attribute? (Use `dir` or tab completion to inspect).
* Categories can be ordered. How do you order *INCOME*?
* There is an `.str` attribute on the string and category columns. What can you do with this attribute? (Use `dir` or tab completion to inspect).
* Uppercase the values in the *TICKET* column.

## Make a Function

In [ ]:
# a glorious function
# add ordered categories to this
def tweak_gss(gss):
    return (gss
      .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
             'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
             'MAJOR1': 'category',
              **{col: 'category' for col in ['SEX', 'RACE', 'BORN', 
                    'INCOME', 'INCOME06', 'HONEST','TICKET']}})
               )

tweak_gss(gss)

## Function Exercise
* Rearrange your notebook. Put the imports, code to load raw data, and tweak function at the top of the notebook. Restart the kernel and validate that your code works.

## Fix Column Names

In [ ]:
# a glorious function
def tweak_gss(gss):
    return (gss
      .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
             'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
             'MAJOR1': 'category',
              **{col: 'category' for col in ['SEX', 'RACE', 'BORN', 
                    'INCOME', 'INCOME06', 'HONEST','TICKET']}})
     .rename(columns={'YEAR': 'year', 'ID': 'year_id', 'AGE':'age', 
          'HRS1': 'hours_worked', 'OCC': 'occupation', 
          'MAJOR1': 'college_major', 'SEX':'sex', 
          'RACE':'race', 'BORN':'born_in_US',
          'INCOME':'income_1970', 'INCOME06': 'income_2006',
          'HONEST':'honesty_rank',
          'TICKET':'traffic_ticket'})
    )

tweak_gss(gss)

## Chain

Chaining is also called "flow" programming. Rather than making intermediate variables, just leverage the fact that most operations return a new object and work on that.

The chain should read like a recipe of ordered steps.

(BTW, this is actually what we did above.)

<div class='alert alert-warning'>
    Hint: Leverage <tt>.pipe</tt> if you can't find a way to chain 😉🐼💪
</div>
    




In [ ]:
# a glorious function
def tweak_gss(gss):
    return (gss
      .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
             'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
             'MAJOR1': 'category',
              **{col: 'category' for col in ['SEX', 'RACE', 'BORN', 
                    'INCOME', 'INCOME06', 'HONEST','TICKET']}})
     .rename(columns={'YEAR': 'year', 'ID': 'year_id', 'AGE':'age', 
          'HRS1': 'hours_worked', 'OCC': 'occupation', 
          'MAJOR1': 'college_major', 'SEX':'sex', 
          'RACE':'race', 'BORN':'born_in_US',
          'INCOME':'income_1970', 'INCOME06': 'income_2006',
          'HONEST':'honesty_rank',
          'TICKET':'traffic_ticket'})
    )

tweak_gss(gss)

In [ ]:
# compare chain to this mess
gss2 = gss.copy()
year = gss.YEAR
year_int = year.astype('uint16')
gss2['year'] = year_int
id = gss.ID
id_int = id.astype('uint16')
gss2['year_id'] = id_int
occ = gss.OCC
occ_int = occ.astype('uint16')
gss2['occupation'] = occ_int

# more of this

In [ ]:
# easy to debug
#  - assign to var (df3)
#  - comment out
#  - pipe to display


from IPython.display import display

def get_var(df, var_name):
    globals()[var_name] = df
    return df

def tweak_gss(gss):
    return (gss
      .pipe(get_var, 'df3')   
     .pipe(lambda df: print(df.shape) or df)                
      .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
             'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
             'MAJOR1': 'category',
              **{col: 'category' for col in ['SEX', 'RACE', 'BORN', 
                    'INCOME', 'INCOME06', 'HONEST','TICKET']}})
     .pipe(lambda df: print(df.shape) or df)                            
     .rename(columns={'YEAR': 'year', 'ID': 'year_id', 'AGE':'age', 
          'HRS1': 'hours_worked', 'OCC': 'occupation', 
          'MAJOR1': 'college_major', 'SEX':'sex', 
          'RACE':'race', 'BORN':'born_in_US',
          'INCOME':'income_1970', 'INCOME06': 'income_2006',
          'HONEST':'honesty_rank',
          'TICKET':'traffic_ticket'})
     .pipe(lambda df: print(df.shape) or df)                            
    )

tweak_gss(gss)

In [ ]:
# inspect intermediate data frame
df3

## Chain Exercise
* Write a function that acccepts a dataframe and an index value. It should print any rows that match the index and return the dataframe that was passed in.
* Use the function with pipe after each step of the chain. Show the rows for index 2 and 64,813.








## Don't Mutate

> "you are missing the point, inplace rarely actually does something inplace, you are thinking that you are saving memory but you are not."
>
> **jreback** - Pandas core dev



https://github.com/pandas-dev/pandas/issues/16529#issuecomment-676518136

* In general, no performance benefits
* Prohibits chaining
* ``SettingWithCopyWarning`` fun


## Don't Apply (if you can)

In [ ]:
# a glorious function
def tweak_gss(gss):
    return (gss
      .astype({'YEAR': 'uint16[pyarrow]', 'ID': 'uint16[pyarrow]', 'OCC': 'uint16[pyarrow]',
             'HRS1': 'uint8[pyarrow]', 'AGE': 'uint8[pyarrow]',
             'MAJOR1': 'category',
              **{col: 'category' for col in ['SEX', 'RACE', 'BORN', 
                    'INCOME', 'INCOME06', 'HONEST','TICKET']}})
     .rename(columns={'YEAR': 'year', 'ID': 'year_id', 'AGE':'age', 
          'HRS1': 'hours_worked', 'OCC': 'occupation', 
          'MAJOR1': 'college_major', 'SEX':'sex', 
          'RACE':'race', 'BORN':'born_in_US',
          'INCOME':'income_1970', 'INCOME06': 'income_2006',
          'HONEST':'honesty_rank',
          'TICKET':'traffic_ticket'})
    )

gss2 = tweak_gss(gss)

In [ ]:
# convert age to months
def to_months(val):
    return val * 12

gss2.age.apply(to_months)

In [ ]:
# this gives the sames results
gss2.age * 12

In [ ]:
%%timeit
gss2.age.apply(to_months)

In [ ]:
%%timeit
gss2.age * 12

In [ ]:
# ~42x slower!
4_590 / 110

In [ ]:
gss.MAJOR1.value_counts()[:20]

In [ ]:
def is_science(val):
    return val in {'Engineering', 'Computer science', 'Biology'}

In [ ]:
%%timeit
# string
gss.MAJOR1.apply(is_science)

In [ ]:
%%timeit
gss.MAJOR1.isin({'Engineering', 'Computer science', 'Biology'})

In [ ]:
%%timeit
# categorical
gss2.college_major.isin({'Engineering', 'Computer science', 'Biology'})

## Apply Exercise
* Make a new column called *minutes_worked* derived the *hours_worked* column.
* Make a new column called *income_ratio*.
  * Convert the income columns to numbers (replace `'No answer'` and `'Refused'` with `np.nan`).
  * Fill in the missing values with the median
  * Divide the 2006 value by 1970 value

## Master Aggregation

Let's compare age by sex by year...🤔

In [ ]:
(gss2
   .groupby('year')
   .mean()
)

In [ ]:
(gss2
   .groupby('year')
   .mean(numeric_only=True)
)

In [ ]:
(gss2
   .groupby('year')
   [['age', 'hours_worked']]
   .mean()
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#plt.style.use('pandas1book') 
sns.set_context('talk')
plt.plot(range(10))

In [ ]:
(gss2
   .groupby('year')
   [['age', 'hours_worked']]
   .median()
   .plot()
)

In [ ]:
(gss2
   .groupby('year')
   [['age', 'hours_worked']]
   #.mean()
   #.median()
   #.std()
   .max()
   .plot()
)

In [ ]:
# add sex
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
   .mean()
   #.median()
   #.std()
   #.max()
   #.plot()
)

In [ ]:
# add sex
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
   .mean()
   #.median()
   #.std()
   #.max()
   .plot()
)

In [ ]:
# unstack
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
   .mean()
   #.median()
   #.std()
   #.max()
   .unstack() 
   .plot()
)

In [ ]:
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
   .mean()
   .unstack()
   .age
)

In [ ]:
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
   .mean()
   .unstack()
   .age
   .plot()
   .legend(bbox_to_anchor=(1,1))
)

In [ ]:
# Let's try looking at hours worked
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
   .mean()
   .unstack()
   .hours_worked
   .plot()
   .legend(bbox_to_anchor=(1,1))
)

In [ ]:
# Multiple aggregates
def second(group):
    return group.iloc[1]
(gss2
   .groupby(['year', 'sex'])
   [['age', 'hours_worked']]
  .agg(['min', 'max', 'mean', second])
   
)

## Aggregation Exercise
* Which occupation has the highest median hours worked?
* Which occupation has the lowest age?
* What is the breakdown of respondents by race for each year?
* Convert the previous to a percentage.
* How many unique occupations are there for each year?
* What is the most popular college_major for each year?
* What is the second most popular college_major for each year?

## Summary

* Correct types save space and enable convenient math, string, and date functionality
* Chaining operations will:
   * Make code readable
   * Remove bugs
   * Easier to debug
* Don't mutate (there's no point). Embrace chaining.
* ``.apply`` is slow for math
* Aggregations are powerful. Play with them until they make sense

Follow on Twitter ``@__mharrison__``

Book giveaway!

In [ ]:
import random
random.randrange(1,13)